# Pipeline de inferencia automatizado para predicción semanal

Este notebook está preparado para ejecutarse automáticamente en un entorno CI/CD (por ejemplo, GitHub Actions) y realizar la predicción semanal de ventas de bollería, almacenando el resultado en Hopsworks.

In [ ]:
# Pipeline de inferencia automatizado para predicción semanal
# Este notebook está preparado para ejecutarse automáticamente en un entorno CI/CD (por ejemplo, GitHub Actions) y realizar la predicción semanal de ventas de bollería, almacenando el resultado en Hopsworks.

import hopsworks
import config

# Conexión al proyecto de Hopsworks
project = hopsworks.login(project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY)


In [ ]:
# Función para automatizar la predicción semanal y su almacenamiento en Hopsworks


def prediccion_semanal_automatizada(
    proyecto_hopsworks,
    nombre_fg="bolleria_predicciones_feature_group",
    version_fg=1,
    nombre_fv="bolleria_predicciones_feature_view",
    version_fv=1,
    nombre_modelo="fleca_bolleria_predictor_next_week",
    version_modelo=1
):
    """
    Automatiza el proceso completo de:
    1. Obtener datos históricos
    2. Preparar features
    3. Predecir siguiente semana
    4. Almacenar predicción en feature store
    """
    import logging
    from datetime import timedelta
    import pandas as pd
    from src.model import transformar_features_target
    from pathlib import Path
    import joblib


    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger('prediccion_semanal')


    try:
        # 1. Conexiones a Hopsworks
        feature_store = proyecto_hopsworks.get_feature_store()
        model_registry = proyecto_hopsworks.get_model_registry()
        logger.info("Conexión a Hopsworks establecida")


        # 2. Obtener datos históricos
        feature_view = feature_store.get_feature_view(name="times_series_bolleria_feature_view", version=1)
        ts_df = feature_view.get_batch_data()
        ts_df = ts_df.sort_values('week_start').reset_index(drop=True)
        logger.info(f"Datos históricos obtenidos: {len(ts_df)} registros")


        # 3. Preparar features
        ts_df = transformar_features_target(
            ts_df,
            lags_list=[1, 52],
            columna_target='base_imponible',
            cols_exogenas=['is_easter', 'is_summer_peak'],
            periodos_adelante=1,
            eliminar_nulos=True,
            return_format='dataframe'
        )
        logger.info("Features preparados correctamente")


        # 4. Cargar modelo
        model = model_registry.get_model(name=nombre_modelo, version=version_modelo)
        model_dir = model.download()
        model_obj = joblib.load(Path(model_dir) / 'xgboost_hopsworks.pkl')
        logger.info(f"Modelo {nombre_modelo} v{version_modelo} cargado correctamente")


        # 5. Obtener la última fecha y predecir siguiente semana
        ultimo_lunes = ts_df['week_start'].max()
        fecha_siguiente = ultimo_lunes + timedelta(days=7)
        last_row = ts_df.iloc[-1].copy()
        last_row['week_start'] = fecha_siguiente
        feature_names = model_obj.feature_names_in_
        X = last_row[feature_names].values.reshape(1, -1)
        prediccion = model_obj.predict(X)[0]


        # 6. Crear DataFrame de resultados
        resultado = pd.DataFrame({
            'week_start': [fecha_siguiente],
            'predicted_base_imponible': [prediccion]
        })
        logger.info(f"Predicción para {fecha_siguiente.date()}: {prediccion:.2f}")


        # 7. Almacenar en feature store
        fg_pred = None
        try:
            fg_pred = feature_store.get_feature_group(name=nombre_fg, version=version_fg)
            logger.info(f"Feature group '{nombre_fg}' existente encontrado")
        except:
            fg_pred = feature_store.create_feature_group(
                name=nombre_fg,
                version=version_fg,
                description="Predicciones semanales de base_imponible para bolleria",
                primary_key=["week_start"],
                online_enabled=False,
                statistics_config={"enabled": True},
                event_time='week_start'
            )
            logger.info(f"Feature group '{nombre_fg}' creado correctamente")


        if fg_pred is not None:
            fg_pred.insert(resultado, write_options={'wait_for_job': True})
            logger.info("Predicción insertada en feature group")
            try:
                fv_pred = feature_store.get_feature_view(name=nombre_fv, version=version_fv)
                logger.info(f"Feature view '{nombre_fv}' existente encontrado")
            except:
                query = fg_pred.select_all()
                fv_pred = feature_store.create_feature_view(
                    name=nombre_fv,
                    version=version_fv,
                    query=query,
                    description="Vista de predicciones semanales de bolleria"
                )
                logger.info(f"Feature view '{nombre_fv}' creado correctamente")
        return resultado
    except Exception as e:
        logger.error(f"Error en predicción automatizada: {e}")
        import traceback
        traceback.print_exc()
        return None


In [ ]:
# Ejecutar la predicción semanal automatizada (solo próxima semana, sin autoregresión)

resultado = prediccion_semanal_automatizada(
    proyecto_hopsworks=project,
    nombre_fg="bolleria_predicciones_feature_group",
    version_fg=1,
    nombre_fv="bolleria_predicciones_feature_view",
    version_fv=1,
    nombre_modelo="fleca_bolleria_predictor_next_week",
    version_modelo=4
)

if resultado is not None:
    print("Predicción realizada y almacenada correctamente:")
    print(resultado)
else:
    print("Error en la predicción semanal automatizada.")